In [1]:
pip install tkinter

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tkinter


In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
import numpy as np
import hashlib
from PIL import Image
import random

def select_image():
    # Let user select an image file
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.webp;*.png;*.jpg;*.jpeg;*.bmp")])
    image_entry.delete(0, tk.END)
    image_entry.insert(0, file_path)

def select_text():
    # Let user select a text file containing the secret message
    file_path = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt")])
    text_entry.delete(0, tk.END)
    text_entry.insert(0, file_path)

def clear_inputs():
    # Clear all input fields
    image_entry.delete(0, tk.END)
    text_entry.delete(0, tk.END)
    password_entry.delete(0, tk.END)

def encrypt_image():
    # Get user inputs from the GUI
    image_path = image_entry.get()
    text_file = text_entry.get()
    password = password_entry.get()
    
    if not image_path or not text_file or not password:
        messagebox.showerror("Error", "All fields are required!")
        return

    # Read the message from the selected text file
    try:
        with open(text_file, "r", encoding="utf-8") as file:
            secret_msg = file.read().strip()
    except FileNotFoundError:
        messagebox.showerror("Error", "Text file not found!")
        return

    # Load the selected image
    try:
        img = Image.open(image_path).convert("RGB")
        pixels = np.array(img, dtype=np.uint8)
    except Exception as e:
        messagebox.showerror("Error", f"Error loading image: {e}")
        return

    # Check if the image can store the message
    max_bytes = pixels.shape[0] * pixels.shape[1] * 3 // 8
    secret_msg += "###END###"  # Mark the end of the message
    binary_msg = ''.join(format(ord(c), '08b') for c in secret_msg)

    if len(binary_msg) > max_bytes * 8:
        messagebox.showerror("Error", "Message too large for image!")
        return

    # Hash the password for secure encryption
    password_hash = hashlib.sha256(password.encode()).hexdigest()
    with open("password_hash.txt", "w") as f:
        f.write(password_hash)
    
    # Log where the message is hidden
    log_file = open("pixel_log_with_gui&stealthMode.txt", "w")
    log_file.write("Bit storage locations (row, col, channel):\n")

    # Randomize pixel selection using the password hash as a seed
    random.seed(password_hash)
    height, width, _ = pixels.shape
    pixel_positions = [(row, col, channel) for row in range(height) for col in range(width) for channel in range(3)]
    random.shuffle(pixel_positions)

    # Embed the message into the image
    idx = 0
    for row, col, channel in pixel_positions:
        if idx < len(binary_msg):
            pixels[row, col, channel] = (int(pixels[row, col, channel]) & ~1) | int(binary_msg[idx])
            log_file.write(f"Bit {binary_msg[idx]} -> Pixel ({row}, {col}, {channel})\n")
            idx += 1

    log_file.close()

    # Save the new image with the hidden message
    encrypted_img = Image.fromarray(pixels)
    encrypted_img.save("encryptedImage_with_gui&stealthmode.png", format="PNG")
    messagebox.showinfo("Success", "Message encrypted successfully in Stealth Mode! Pixel log saved.")

def decrypt_image():
    # Get password input for decryption
    password = password_entry.get()
    if not password:
        messagebox.showerror("Error", "Password required for decryption!")
        return

    # Check if the stored password matches the entered password
    try:
        with open("password_hash.txt", "r") as f:
            stored_hash = f.read().strip()
    except FileNotFoundError:
        messagebox.showerror("Error", "Password file not found!")
        return

    password_hash = hashlib.sha256(password.encode()).hexdigest()
    if password_hash != stored_hash:
        messagebox.showerror("Error", "Incorrect password!")
        return

    # Load the encrypted image
    image_path = image_entry.get()
    if not image_path:
        messagebox.showerror("Error", "Encrypted image path required!")
        return

    try:
        img = Image.open(image_path).convert("RGB")
        pixels = np.array(img, dtype=np.uint8)
    except Exception as e:
        messagebox.showerror("Error", f"Error loading image: {e}")
        return

    # Use the password hash to get the same random pixel order
    random.seed(password_hash)
    height, width, _ = pixels.shape
    pixel_positions = [(row, col, channel) for row in range(height) for col in range(width) for channel in range(3)]
    random.shuffle(pixel_positions)

    # Extract the binary message from the image
    binary_msg = ""
    for row, col, channel in pixel_positions:
        binary_msg += str(pixels[row, col, channel] & 1)

    # Convert binary data back to text
    chars = [binary_msg[i:i+8] for i in range(0, len(binary_msg), 8)]
    message = ''.join(chr(int(c, 2)) for c in chars)
    
    # Extract message until the end marker
    if "###END###" in message:
        message = message[:message.index("###END###")]
        messagebox.showinfo("Decrypted Message", message)
    else:
        messagebox.showerror("Error", "No hidden message found!")

root = tk.Tk()
root.title("Steganography Tool")

# Create GUI elements for file selection and encryption options
tk.Label(root, text="Image File:").grid(row=0, column=0, padx=10, pady=5)
image_entry = tk.Entry(root, width=60)
image_entry.grid(row=0, column=1, padx=10, pady=5)
tk.Button(root, text="Browse", command=select_image).grid(row=0, column=2, padx=10, pady=5)

tk.Label(root, text="Text File:").grid(row=1, column=0, padx=10, pady=5)
text_entry = tk.Entry(root, width=60)
text_entry.grid(row=1, column=1, padx=10, pady=5)
tk.Button(root, text="Browse", command=select_text).grid(row=1, column=2, padx=10, pady=5)

tk.Label(root, text="Password:").grid(row=2, column=0, padx=10, pady=5)
password_entry = tk.Entry(root, width=60, show="*")
password_entry.grid(row=2, column=1, padx=10, pady=5)

tk.Button(root, text="Encrypt Image", command=encrypt_image).grid(row=3, column=1, padx=10, pady=10)
tk.Button(root, text="Decrypt Message", command=decrypt_image).grid(row=4, column=1, padx=10, pady=10)
tk.Button(root, text="Clear Fields", command=clear_inputs).grid(row=5, column=1, padx=10, pady=10)

root.mainloop()
